In [1]:
!pip install matplotlib_inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import scipy.stats as ss
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge, LinearRegression

In [3]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')
import scipy.stats as ss
from scipy import optimize
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint
import pandas as pd
from scipy.integrate import quad
from itertools import product
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

**Задание 1**

In [6]:
data = np.array([28, 26, 33, 24, 34, -44, 27, 16, 40, -2, 29,
        22, 24, 21, 25, 30, 23, 29, 31, 19, 24, 20,
        36, 32, 36, 28, 25, 21, 28, 29, 37, 25, 28,
        26, 30, 32, 36, 26, 30, 22, 36, 23, 27, 27,
        28, 27, 31, 27, 26, 33, 26, 32, 32, 24, 39,
        28, 24, 25, 32, 25, 29, 27, 28, 29, 16, 23])


n = len(data)
N = 10000

sigma_2_sample = (n - 1) * data.std(ddof=1)**2 / ss.chi2(n-1).rvs(size=N, random_state=42)

normal_sample = ss.norm().rvs(size=N, random_state=42)

mu_sample = normal_sample * np.sqrt(sigma_2_sample / n) + data.mean()

alpha = 0.05
np.quantile(mu_sample, alpha/2), np.quantile(mu_sample, 1 - alpha/2)

(23.551581550110072, 28.824085574962552)

**Задание 2**

In [7]:
data = np.array([28, 26, 33, 24, 34, -44, 27, 16, 40, -2, 29,
        22, 24, 21, 25, 30, 23, 29, 31, 19, 24, 20,
        36, 32, 36, 28, 25, 21, 28, 29, 37, 25, 28,
        26, 30, 32, 36, 26, 30, 22, 36, 23, 27, 27,
        28, 27, 31, 27, 26, 33, 26, 32, 32, 24, 39,
        28, 24, 25, 32, 25, 29, 27, 28, 29, 16, 23])

n = len(data)
N = 20000
sigma_2_sample = (n - 1) * ((data**2).sum()/n - data.mean()**2) / ss.chi2(n-1).rvs(size=N, random_state=42)
normal_sample = ss.norm().rvs(size=N, random_state=42)
mu_sample = normal_sample * np.sqrt(sigma_2_sample / n) + data.mean()

sample = ss.norm().rvs(size=(N,n), random_state=37)
x_samples = np.diag(np.sqrt(sigma_2_sample)) @ sample + np.array([mu_sample for _ in range(n)]).T
xmin_sample = x_samples.min(axis=-1)

alpha = 0.05
np.quantile(xmin_sample, alpha/2), np.quantile(xmin_sample, 1 - alpha/2)

(-11.68469123931682, 10.034396577715343)

**Задание 3**

In [8]:
import numpy as np
import scipy.stats as ss

p1, p2 = 0.3, 0.7                                                # p(i)
mix = ss.rv_discrete(name='mix', values=([1, 2], [p1, p2]))
px1 = ss.multivariate_normal([0, 0], [[1, 0.5], [0.5, 1]])       # p(x|1)
px2 = ss.multivariate_normal([2.5, 0], [[1, -0.5], [-0.5, 1]])   # p(x|2)
    
def make_data(n):
    y = mix.rvs(size=n)
    X = np.zeros((n, 2))
    X[y == 1] = px1.rvs(size=n)[y == 1]
    X[y == 2] = px2.rvs(size=n)[y == 2]
    return X, y


def a(x):
  p = [0.3 * px1.pdf(x), 0.7 * px2.pdf(x)]
  return np.argmax(p) + 1


N = 100000
X, y = make_data(N)
([a(x) for x in X] != y).mean()

0.08868